In [1]:
import pandas as pd
import os
from mlflow.tracking import MlflowClient


from utils import get_params_and_metrics
from utils import METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS

NOTEBOOK_URI="sqlite:////home/guillermo/Documents/SINC/Research/RNA/AEseq2seq/mlruns.db"
SINC_URI="sqlite:////home/gkulemeyer/Documents/Repos/AEseq2seq/mlruns.db"

# Conectar al backend store
client = MlflowClient(tracking_uri=SINC_URI)
experiments = client.search_experiments()
for i,exp in enumerate(experiments):
    print(f"i: {i}, ID: {exp.experiment_id}, Nombre: {exp.name}, Estado: {exp.lifecycle_stage}")

i: 0, ID: 28, Nombre: Unet_fixed_models_ablation, Estado: active
i: 1, ID: 27, Nombre: Unet_fixed_nc4_nc8_noise_0_1, Estado: active
i: 2, ID: 26, Nombre: Unet_srp_nc4_nc8_noise_0_1, Estado: active
i: 3, ID: 25, Nombre: Unet_nc4_nc8_noise_0_1, Estado: active
i: 4, ID: 24, Nombre: Unet_paper_noise_0_1, Estado: active
i: 5, ID: 23, Nombre: tiny_Unet_noise_exp3, Estado: active
i: 6, ID: 22, Nombre: tiny_Unet_noise_exp2, Estado: active
i: 7, ID: 21, Nombre: tiny_Unet_noise_exp, Estado: active
i: 8, ID: 20, Nombre: paper_based_unet_batch_test, Estado: active
i: 9, ID: 19, Nombre: paper_based_unet_noise_test, Estado: active
i: 10, ID: 18, Nombre: paper_based_unet_pc59_2, Estado: active
i: 11, ID: 17, Nombre: paper_based_unet_pc59_test, Estado: active
i: 12, ID: 16, Nombre: paper_based_unet_features_raw, Estado: active
i: 13, ID: 15, Nombre: paper_based_unet_features_x4_x8, Estado: active
i: 14, ID: 14, Nombre: paper_based_unet_fam_split, Estado: active
i: 15, ID: 13, Nombre: paper_based_unet_

In [30]:
pd.options.display.max_columns = 999

In [8]:
n = 0
name = experiments[n].name
id = experiments[n].experiment_id

print(f"Nombre: {name}, ID: {id}, Estado: {experiments[n].lifecycle_stage}")

df_arc = get_params_and_metrics(client, id, ARCHITECTURE)
for col in ARCHITECTURE:
    print(f"{col}: {df_arc[col].unique()}")

Nombre: Unet_fixed_models_ablation, ID: 28, Estado: active
arc_embedding_dim: ['4']
arc_features: ['[4, 4, 4, 4, 8, 8]' '[4, 4, 4, 4, 8]' '[4, 4, 4, 4]' '[4, 4, 4, 8, 8]'
 '[4, 4, 4, 8]' '[4, 4, 4]' '[4, 4, 8, 8]' '[4, 4, 8]' '[4, 4]'
 '[4, 8, 8]' '[4, 8]' '[4]' '[4, 8, 16, 32, 64]']
arc_encoder_blocks: ['5' '4' '3' '2' '1' '0']
arc_initial_volume: ['512']
arc_latent_volume: ['32.0' '64.0' '128.0' '256.0' '512.0']
arc_num_conv: ['2']
arc_pool_mode: ['max' 'avg']
arc_up_mode: ['upsample' 'transpose']
arc_addition: ['cat' 'sum']
arc_skip: ['1' '0']


In [9]:
# METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS
arc = [ 
    "arc_features",
    "arc_encoder_blocks",
    "arc_latent_volume",
    "arc_skip",
    'arc_pool_mode',
    'arc_up_mode',
    'arc_addition',
    'arc_skip'
]

df_metrics = get_params_and_metrics(client, id, ["test_file"] + arc + METRICS)
print(f"df shape: {df_metrics.shape}")
df_metrics.sample(6)

df shape: (195, 22)


,run_name,test_file,arc_features,arc_encoder_blocks,arc_latent_volume,arc_skip,arc_pool_mode,arc_up_mode,arc_addition,arc_skip,...,test_loss,valid_F1,train_F1,test_F1,train_Accuracy,valid_Accuracy,test_Accuracy,train_Accuracy_seq,valid_Accuracy_seq,test_Accuracy_seq
80,model_n4-2-n8-0-p-max-up-upsample-add-cat-skip1,test_0.csv,"[4, 4, 4]",2,128.0,1,max,upsample,cat,1,...,0.054293,0.767646,0.778750,0.757876,0.778750,0.767646,0.757876,0.0,0.0,0.0
48,model_n4-2-n8-2-p-max-up-upsample-add-cat-skip1,test_0.csv,"[4, 4, 4, 8, 8]",4,64.0,1,max,upsample,cat,1,...,0.024649,0.995772,0.997627,0.998614,0.997627,0.995772,0.998614,0.0,0.0,0.0
83,model_n4-2-n8-0-p-max-up-upsample-add-sum-skip0,test_0.csv,"[4, 4, 4]",2,128.0,0,max,upsample,sum,0,...,0.147471,0.370735,0.388036,0.362529,0.388036,0.370735,0.362529,0.0,0.0,0.0
99,model_n4-1-n8-2-p-max-up-upsample-add-sum-skip0,test_0.csv,"[4, 4, 8, 8]",3,128.0,0,max,upsample,sum,0,...,0.145349,0.384613,0.415881,0.365800,0.415881,0.384613,0.365800,0.0,0.0,0.0
193,model_n4-0-n8-0-p-avg-uptranspose-addsum-skip1,test_0.csv,"[4, 8, 16, 32, 64]",4,512.0,1,avg,transpose,sum,1,...,0.050949,0.767482,0.779187,0.758259,0.779187,0.767482,0.758259,0.0,0.0,0.0
116,model_n4-1-n8-1-p-max-up-transpose-add-cat-skip1,test_0.csv,"[4, 4, 8]",2,256.0,1,max,transpose,cat,1,...,0.006488,0.999591,0.999198,0.999474,0.999198,0.999591,0.999474,0.0,0.0,0.0


In [12]:
# METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS
cols = [ 
    "arc_features",
    "arc_encoder_blocks",
    "arc_latent_volume",
    "arc_num_params",
    "arc_skip",
    'arc_pool_mode',
    'arc_up_mode',
    'arc_addition',
    'arc_skip'
]

df_metrics = get_params_and_metrics(client, id,  cols + METRICS)
print(f"df shape: {df_metrics.shape}")
df_metrics.sample(6)

df shape: (195, 22)


,run_name,arc_features,arc_encoder_blocks,arc_latent_volume,arc_num_params,arc_skip,arc_pool_mode,arc_up_mode,arc_addition,arc_skip,...,test_loss,valid_F1,train_F1,test_F1,train_Accuracy,valid_Accuracy,test_Accuracy,train_Accuracy_seq,valid_Accuracy_seq,test_Accuracy_seq
105,model_n4-1-n8-2-p-avg-up-upsample-add-cat-skip0,"[4, 4, 8, 8]",3,128.0,1748,0,avg,upsample,cat,0,...,0.143941,0.395687,0.420013,0.376703,0.420013,0.395687,0.376703,0.0,0.0,0.0
139,model_n4-1-n8-0-p-avg-up-upsample-add-sum-skip0,"[4, 4]",1,256.0,380,0,avg,upsample,sum,0,...,0.116405,0.517051,0.535205,0.518072,0.535205,0.517051,0.518072,0.0,0.0,0.0
28,model_n4-3-n8-1-p-avg-up-transpose-add-cat-skip1,"[4, 4, 4, 4, 8]",4,64.0,1764,1,avg,transpose,cat,1,...,0.041797,0.999267,0.997245,0.999194,0.997245,0.999267,0.999194,0.0,0.0,0.0
42,model_n4-3-n8-0-p-avg-up-upsample-add-sum-skip1,"[4, 4, 4, 4]",3,64.0,920,1,avg,upsample,sum,1,...,0.059760,0.775165,0.788789,0.764632,0.788789,0.775165,0.764632,0.0,0.0,0.0
180,model_n4-0-n8-0-p-max-up-transpose-add-cat-skip1,[4],0,512.0,140,1,max,transpose,cat,1,...,0.032805,0.997550,0.997625,0.998793,0.997625,0.997550,0.998793,0.0,0.0,0.0
165,model_n4-0-n8-1-p-max-up-transpose-add-cat-skip0,"[4, 8]",1,512.0,696,0,max,transpose,cat,0,...,0.098759,0.717563,0.721960,0.706237,0.721960,0.717563,0.706237,0.0,0.0,0.0


In [13]:
EXPERIMENT_PATH = os.path.join("Unet_selection", name)
os.makedirs(EXPERIMENT_PATH, exist_ok=True)
df_metrics.to_csv(os.path.join(EXPERIMENT_PATH, "metrics.csv"), index=False)

In [7]:
cols = ['batch_size', 'test_loss',	'test_F1',	'test_Accuracy']
df_metrics = get_params_and_metrics(client, id, ["test_file"] + cols)
print(f"df shape: {df_metrics.shape}")
df_metrics.sample(6)

EXPERIMENT_PATH = os.path.join("Unet_selection", name)
os.makedirs(EXPERIMENT_PATH, exist_ok=True)
df_metrics.to_csv(os.path.join(EXPERIMENT_PATH, "batch_test.csv"), index=False)

falta train_file
falta valid_file
df shape: (133, 6)
